In [1]:
import nrrd
import numpy as np
import os
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from skimage.segmentation import mark_boundaries
import cv2
from scipy import ndimage as ndi
from helper import *
import graph_tool.all as gt
import plotly.graph_objects as go
import time

current_directory = os.getcwd()
label_path = f"{current_directory}/data/label/manual_1_label.nrrd"
raw_data_path = f"{current_directory}/data/raw/manual_1_raw.nrrd"

mask_data, mask_header = nrrd.read(label_path)
raw_data, raw_header = nrrd.read(raw_data_path)

In [2]:
#rotate the data for best cut angle and calculate reduced data representations
axis = 1 #0 for x, 1 for y and 2 for z
k = 1 # number of times to rotate the array
mask_data = np.rot90(mask_data, k, axes=(axis, (axis+1)%3))
raw_data = np.rot90(raw_data, k, axes=(axis, (axis+1)%3))

reduced_mask_data = coarsen_image(mask_data, 3)
reduced_raw_data = coarsen_image(raw_data, 3)
#[0] is full res, each further index is 2x lower res, 256, 128, 64, 32

In [3]:
#create reduced graph
res_index = 2
rotated_directed_graph, r_src, r_tgt, rotated_weights, x_pos, y_pos, z_pos = create_masked_directed_energy_graph_from_mask(reduced_mask_data[res_index])

64 64 64


In [4]:
def calculate_seam_iter(directed_graph, src, tgt, weights, test_size, x_pos, y_pos, z_pos):
    # Compute the residual capactiy of the edges
    stime = time.time()
    res = gt.boykov_kolmogorov_max_flow(directed_graph, src, tgt, weights) #time complexity: edges * vertices^2 * abs(min cut) 
    print("Time taken to calculate max flow boykov:", time.time()-stime)
    # sptime = time.time()
    # res = gt.push_relabel_max_flow(directed_graph, src, tgt, weights) #time complexity: vertices^3 <- scales better for our graph
    # print("Time taken to calculate max flow push relabel:", time.time()-sptime)
    stime = time.time()
    #use the residual graph to get the max flow
    flow = sum(weights[e] - res[e] for e in tgt.in_edges())
    # print("The maximum flow from source to sink is:", flow)
    print("Time taken to calculate max flow:", time.time()-stime, "flow:", flow)
    stime = time.time()
    # Determine the minimum cut partition
    part = gt.min_st_cut(directed_graph, src, weights, res)
    # print("The number of vertices in the partiton:", sum(part))
    print("Time taken to calculate max flow and min cut:", time.time()-stime)
    # Find the boundary vertices
    stime = time.time()
    boundary_vertices = find_boundary_vertices(np.array(directed_graph.get_edges()), part)
    # print("Number of boundary vertices:", len(boundary_vertices))
    print("Time taken to calculate seam:", time.time()-stime)

    shape = (test_size, test_size, test_size)

    # Convert the boundary vertices to a 3D array
    stime = time.time()
    boundary_array = boundary_vertices_to_array_masked(boundary_vertices, shape, 'x', x_pos, y_pos, z_pos)
    # print("Boundary points marked:", np.sum(boundary_array))
    print("Time taken to convert boundary vertices to array:", time.time()-stime)

    return boundary_array, weights, flow

In [5]:
boundary_array, rotated_weights, flow = calculate_seam_iter(rotated_directed_graph, r_src, r_tgt, rotated_weights, reduced_mask_data[res_index].shape[0], x_pos, y_pos, z_pos)
b_arr_up = upscale_and_dilate_3d(boundary_array, upscale_factor=4, dilation_amount=1)
print(b_arr_up.shape)

Time taken to calculate max flow boykov: 4.14008903503418
Time taken to calculate max flow: 0.04186892509460449 flow: 8848
Time taken to calculate max flow and min cut: 0.09170913696289062
edges that cross the partition: 1540352
boundary vertices: 6523
Time taken to calculate seam: 0.01726698875427246
Time taken to convert boundary vertices to array: 0.21114301681518555
(256, 256, 256)


In [6]:
# copying and masking the high res data for masked graph creation
res_index = 0
masked_array = reduced_mask_data[res_index].copy()
print(masked_array.shape, b_arr_up.shape)
masked_array[b_arr_up == 0] = -1

masked_graph, masked_src, masked_sink, masked_weights, x_pos, y_pos, z_pos = create_masked_directed_energy_graph_from_mask(masked_array)

(256, 256, 256) (256, 256, 256)
256 256 256


In [9]:
boundary_array, masked_weights, flow = calculate_seam_iter(masked_graph, masked_src, masked_sink, masked_weights, masked_array.shape[0], x_pos, y_pos, z_pos)

Time taken to calculate max flow boykov: 2.8004941940307617
Time taken to calculate max flow push relabel: 3.210710048675537
Time taken to calculate max flow: 0.655430793762207 flow: 121111
Time taken to calculate max flow and min cut: 6.2647340297698975
edges that cross the partition: 2436094
boundary vertices: 102487
Time taken to calculate seam: 0.06717801094055176
Time taken to convert boundary vertices to array: 12.855855941772461
